
# Landsat 7/8 — Composite Median (RGB) & Water (MNDWI) on Google Earth Engine

**Autor:** Luis Gabriel Hernández Ojeda  
**Descripción:** Este cuaderno crea un *composite median* Landsat 7/8 (RGB) y una capa de **agua** usando **MNDWI**, visualiza ambas en un mapa interactivo (Folium) y deja una celda opcional para exportar los resultados a Google Drive.

**Puntos clave:**  
- Filtrado por fechas, meses y nubosidad (CLOUD\_COVER).  
- Enmascaramiento de nubes con `QA_PIXEL`.  
- Cálculo de **MNDWI** (GREEN − SWIR1) para realzar cuerpos de agua.  
- Visualización RGB con **reflectancia** (escala: `0.0000275`, offset: `-0.2`) y/o stretch por percentiles.  
- Todo el flujo funciona tanto en Colab como en notebooks locales autenticados con Earth Engine.


In [ ]:

# === 1) Dependencias =========================================================
# Si estás en Google Colab, normalmente ya están instaladas.
# Si ejecutas localmente y hay error de import, descomenta lo siguiente:
# !pip install earthengine-api folium

import ee
import folium
from typing import Dict, Any, Tuple


In [ ]:

# === 2) Autenticación con Earth Engine ======================================
# En Colab: ejecuta ee.Authenticate() cuando lo pida, luego ee.Initialize().
# En local (Jupyter): debes tener credenciales configuradas.
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()
    
print('EE inicializado correctamente.')


In [ ]:

# === 3) Parámetros del análisis =============================================
# Define tu región de interés (ROI). Opciones:
#   A) Un punto y un buffer (km)
#   B) Un polígono explícito (lista de coordenadas)
#   C) Cargar un asset: ee.FeatureCollection('users/tu_usuario/tu_asset').geometry()

# ---- Opción A: punto + buffer (editables) ----
center_lon, center_lat = -74.65, 8.65   # Ejemplo: Mojana
buffer_km = 30                           # Radio en km para el ROI

point = ee.Geometry.Point([center_lon, center_lat])
roi   = point.buffer(buffer_km * 1000).bounds()  # bounds() para una caja simple

# ---- Fechas / meses / nubosidad ----
date_start = '2019-01-01'
date_end   = '2024-12-31'
valid_months = list(range(1, 13))   # 1..12
max_cloud_cover = 20                # CLOUD_COVER (%) máximo por escena

# ---- Parámetros de visualización ----
# OJO: el flujo tiene dos opciones de visualización (ver más abajo):
#  A) Reflectancia escalada [0..0.3]
#  B) DN crudo con stretch alto [~5000..15000]
use_reflectance = True  # True = opción A; False = opción B


In [ ]:

# === 4) Utilidad de mapa (Folium) ===========================================
def mapdisplay(center_latlon: Tuple[float, float],
               tile_dict: Dict[str, Dict[str, Any]],
               zoom_start: int = 9,
               basemap: str = 'OpenStreetMap') -> folium.Map:
    """Crea un mapa Folium y agrega cada capa de GEE (getMapId) como TileLayer.
    :param center_latlon: (lat, lon) para centrar el mapa
    :param tile_dict: dict {'nombre': getMapId(vis_params), ...}
                      getMapId retorna un dict con 'tile_fetcher'.url_format
    :param zoom_start: zoom inicial
    :param basemap: nombre del mapa base
    """
    lat, lon = center_latlon
    m = folium.Map(location=[lat, lon], tiles=basemap, zoom_start=zoom_start)
    for name, v in tile_dict.items():
        if isinstance(v, dict) and 'tile_fetcher' in v:
            folium.TileLayer(
                tiles=v['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(m)
        else:
            # Si no es un dict de getMapId, intentar como GeoJSON (no usado aquí)
            try:
                folium.GeoJson(data=v, name=name).add_to(m)
            except Exception:
                pass
    folium.LayerControl(collapsed=False).add_to(m)
    return m


In [ ]:

# === 5) Preprocesamiento Landsat y cálculo de MNDWI =========================
def _mask_clouds_landsat_l2(image: ee.Image) -> ee.Image:
    """Enmascara nubes/sombra basándose en QA_PIXEL (bits 3,5,7)."""
    qa = image.select('QA_PIXEL')
    mask_bits = (1 << 3) | (1 << 5) | (1 << 7)  # cloud shadow, cloud, cirrus
    mask = qa.bitwiseAnd(mask_bits).eq(0)
    return image.updateMask(mask)

def preprocess_l7(image: ee.Image) -> ee.Image:
    """Renombra bandas de Landsat 7 L2 y agrega MNDWI (GREEN-SWIR1)."""
    image = _mask_clouds_landsat_l2(image)
    # MNDWI = (GREEN - SWIR1) / (GREEN + SWIR1)
    mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')
    return (image.addBands(mndwi)
                 .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','MNDWI'])
                 .rename(['BLUE','GREEN','RED','NIR','SWIR','SWIR2','MNDWI'])
                 .copyProperties(image, ['system:time_start']))

def preprocess_l8(image: ee.Image) -> ee.Image:
    """Renombra bandas de Landsat 8 L2 y agrega MNDWI (GREEN-SWIR1)."""
    image = _mask_clouds_landsat_l2(image)
    mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')
    return (image.addBands(mndwi)
                 .select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','MNDWI'])
                 .rename(['BLUE','GREEN','RED','NIR','SWIR','SWIR2','MNDWI'])
                 .copyProperties(image, ['system:time_start']))

def to_reflectance(img: ee.Image) -> ee.Image:
    """Escala bandas ópticas Landsat L2 a reflectancia: DN*0.0000275 - 0.2."""
    # Solo a ópticas; MNDWI ya está en [-1..1]
    optical = img.select(['BLUE','GREEN','RED','NIR','SWIR','SWIR2'])                  .multiply(0.0000275).add(-0.2)
    return img.addBands(optical, overwrite=True)


In [ ]:

# === 6) Construcción del composite =========================================
def build_l7_l8_composite(roi: ee.Geometry,
                           date_start: str,
                           date_end: str,
                           valid_months: list,
                           max_cloud_cover: int,
                           composite: str = 'median') -> ee.Image:
    """Crea un composite Landsat 7/8 filtrado por fechas, meses y nubosidad.
    composite: 'median' para RGB mediano; 'ndwi' para qualityMosaic(MNDWI).
    """
    l7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
            .filterBounds(roi)
            .filterDate(date_start, date_end)
            .filter(ee.Filter.calendarRange(valid_months[0], valid_months[-1], 'month'))
            .filter(ee.Filter.lte('CLOUD_COVER', max_cloud_cover))
            .map(preprocess_l7))

    l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            .filterBounds(roi)
            .filterDate(date_start, date_end)
            .filter(ee.Filter.calendarRange(valid_months[0], valid_months[-1], 'month'))
            .filter(ee.Filter.lte('CLOUD_COVER', max_cloud_cover))
            .map(preprocess_l8))

    l78 = l7.merge(l8)

    if composite.lower() == 'median':
        return l78.median().clip(roi)
    elif composite.lower() == 'ndwi':
        return l78.qualityMosaic('MNDWI').clip(roi)
    else:
        return l78.median().clip(roi)  # default seguro


In [ ]:

# === 7) Ejecutar y preparar visualización ===================================
compos = build_l7_l8_composite(
    roi=roi,
    date_start=date_start,
    date_end=date_end,
    valid_months=valid_months,
    max_cloud_cover=max_cloud_cover,
    composite='median'  # 'median' para RGB; cambia a 'ndwi' si quieres qualityMosaic(MNDWI)
)

# Visualización: reflectancia o DN crudo
if use_reflectance:
    compos_for_rgb = to_reflectance(compos)
    vis_rgb = {'bands': ['RED','GREEN','BLUE'], 'min': 0.0, 'max': 0.3, 'gamma': 1.2}
else:
    compos_for_rgb = compos
    vis_rgb = {'bands': ['RED','GREEN','BLUE'], 'min': 5000, 'max': 15000, 'gamma': 1.2}

# Visualización MNDWI (agua). Ajusta min/max según escena si lo necesitas.
vis_mndwi = {'bands': ['MNDWI'], 'min': -0.1, 'max': 1.0}

# Crea diccionario de capas para Folium (getMapId)
tiles = {
    'composite_median': compos_for_rgb.getMapId(vis_rgb),
    'agua_mndwi': compos.getMapId(vis_mndwi)
}

# Mostrar mapa
m = mapdisplay(center_latlon=(center_lat, center_lon), tile_dict=tiles, zoom_start=9)
m


In [ ]:

# === 8) (Opcional) Stretch por percentiles para RGB =========================
# Ejecuta esta celda si quieres calcular percentiles sobre el ROI y actualizar la visual.
# Nota: reduceRegion(getInfo) trae resultados al cliente y puede tardar según el área.
# Si el ROI es muy grande, considera usar una submuestra o escala más alta.
try:
    p = compos_for_rgb.select(['RED','GREEN','BLUE']).reduceRegion(
        reducer=ee.Reducer.percentile([2, 98]),
        geometry=roi, scale=30, maxPixels=1e9
    ).getInfo()

    vis_auto = {
        'bands': ['RED','GREEN','BLUE'],
        'min': [p['RED_p2'], p['GREEN_p2'], p['BLUE_p2']],
        'max': [p['RED_p98'], p['GREEN_p98'], p['BLUE_p98']],
        'gamma': 1.1
    }

    tiles_auto = {
        'composite_median_auto': compos_for_rgb.getMapId(vis_auto),
        'agua_mndwi': compos.getMapId({'bands': ['MNDWI'], 'min': -0.1, 'max': 1.0})
    }

    m2 = mapdisplay(center_latlon=(center_lat, center_lon), tile_dict=tiles_auto, zoom_start=9)
    m2
except Exception as e:
    print('No se pudo calcular percentiles (posible ROI muy grande o problema de credenciales).')
    print('Detalle:', e)


In [ ]:

# === 9) (Opcional) Exportar imágenes a Google Drive =========================
# Descomenta y ejecuta si deseas exportar el composite RGB y/o MNDWI.
# NOTA: La exportación se gestiona en la "Tasks" (panel de Earth Engine Code Editor)
# o en la API según el entorno.

# # Exportar composite RGB (reflec. si usas_reflectance=True)
# task_rgb = ee.batch.Export.image.toDrive(
#     image=compos_for_rgb.select(['RED','GREEN','BLUE']),
#     description='L78_composite_median_RGB',
#     folder='MOJANA',  # cambia si deseas otra carpeta en Drive
#     fileNamePrefix='L78_composite_median_RGB',
#     region=roi,
#     scale=30,
#     maxPixels=1e13
# )
# task_rgb.start()

# # Exportar MNDWI
# task_mndwi = ee.batch.Export.image.toDrive(
#     image=compos.select(['MNDWI']),
#     description='L78_MNDWI',
#     folder='MOJANA',
#     fileNamePrefix='L78_MNDWI',
#     region=roi,
#     scale=30,
#     maxPixels=1e13
# )
# task_mndwi.start()

# print('Exportaciones iniciadas (verifica el panel de Tareas en tu entorno de EE).')



---

## Notas finales
- Si el **RGB se ve muy claro u oscuro**, alterna entre `use_reflectance=True` (0–0.3) y `False` (DN 5000–15000) o usa la celda de **percentiles**.  
- Para áreas muy grandes, el cálculo de percentiles puede tardar; filtra tu ROI o aumenta la escala.  
- Evita exponer credenciales (tokens) en el código y usa `ee.Authenticate()`/`ee.Initialize()` en tu sesión.  

Cualquier mejora o bugfix, ¡bienvenida vía Pull Request en GitHub!
